# Parsing HTML with BeautifulSoup

Once we've fetched the HTML using requests, the next step is to parse the HTML into a data structure that Python can work with. To do this, we'll use a Python library called [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), which is aliased to `bs4` in the package management system we're using.

Our goal is to scrape each row of that [table of inmates on death row in Texas](https://www.tdcj.state.tx.us/death_row/dr_offenders_on_dr.html) into a Python data structure called a _list_.

`'https://www.tdcj.state.tx.us/death_row/dr_offenders_on_dr.html'`

Along the way, we'll also encounter a Python data structure called a dictionary, and a Python statement called a `for loop`. How much fun are _we_ having!

👉 For more details on lists, dictionaries and for loops, [see this notebook](../_Python%20syntax%20cheat%20sheet.ipynb).

So to start off, we need to import our dependencies -- `requests` to fetch the HTML and `bs4` to parse it:

Next, let's fetch the page, save it to a variable and make sure we've got the HTML:

In [ ]:
# fetch the page


In [ ]:
# have a look at the .text attribute


Now we can hand off that HTML, which lives in `dr_page.text`, to a BeautifulSoup object, which will parse the HTML into something we can more easily navigate. We'll save the result as a new variable, `soup`.

Here's how to do that:

In [ ]:
# turn it into soup


The `'html.parser'` bit specifies _how_ we want to parse the text ([more details here, if you're interested](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser)).

If you run a cell to return `soup`, here's what you get:

In [ ]:
# let's take a look at the soup


Doesn't look too different from `dr_page.text`. But what's happened under the hood is that `bs4` has parsed that raw chunk of HTML into a navigable tree that we can search through to target specific elements.

In [ ]:
# what type of thing is dr_page.text?


In [ ]:
# what type of thing is soup?


Now we can target the elements we're interested in scraping. If you don't have the page open already, [open the page in a new tab](https://www.tdcj.texas.gov/death_row/dr_offenders_on_dr.html) and right-click on the table of data we want to scrape and select "Inspect" (assuming you're in Chrome -- Firefox has similar developer tools).

If you clicked on one of the header cells, you'll see that the text is inside a `th` tag, which stands for "table header."

`<th scope="col" abbr="tdcj number">TDCJ<br>Number</th>`

HTML elements are nested -- if you work your way up a bit, you'll see that this `th` is a child element to a `tr` tag, which stands for "table row." That parent `tr` tag contains all of the headers in that table.

Moving up one more level, we come to what we're looking for: the `table` element. (For more details on HTML tables, [check out this explainer](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/table).)

`<table class="tdcj_table indent" style="width:98%">`

(Another way to check this out would be to hit Ctrl+U to view source on the page, hit Ctrl+F and search for "TDCJ Number," which is the first cell in the table.)

BeautifulSoup offers several ways to target elements on a page -- for this one, we could say:
- Find the first table on the page (or whatever number it is -- you'd want to Ctrl-F and search for `<table` to see how many show up), or
- Find the table with the CSS class "tdcj_table", or
- Find the table with the style "width:98%", or
- Some combination of those

For this exercise, let's use the `class` attribute to target this table. We'll use the BeautifulSoup `find()` method to look for a table and pass it a _dictionary_ with the class information. While we're at it, we'll save the results to a new variable called `table`.

(Dictionaries are Python data structures that match keys to values -- you can read more about them [here](../_Python%20syntax%20cheat%20sheet.ipynb#Dictionaries).)

In [ ]:
# use the class name to find the correct table


In [ ]:
# let's take a look at the `table` variable

Perfect! Next, we want to get a _list_ of every row in the table. (The tag for a table row, again, is `tr`.) To do this, we'll use a different BeautifulSoup method called `find_all()`, which returns a Python list of elements that match the criteria. In human words, we're saying: Go to the table we just targeted and find all of the `tr` tags within.

Save the results as a new variable, `rows`.

In [ ]:
# find_all() table rows


In [ ]:
# take a look at the rows


It's a little hard to see, but we've got ourselves a list (lists are enclosed in square brackets `[]`) of rows. To see how many items are in the list, we can use the `len()` function:

In [ ]:
# how many rows are there? use len()


We could also use a `for loop` to check out each item a little more closely.

A `for loop` starts with the word `for` (lowercase), then a placeholder value that will stand in for each item in the list as we loop over it, then the word `in` (lowercase), then the name of the list we're looping over (`rows`, in this case), then a colon.

The lines underneath the for loop need to be indented with the same number of spaces (or you could use tabs, if you're a monster). Jupyter defaults to 4 spaces. Everything in that indended code block will be applied to each item in the list as we loop over it.

Let's just `print()` each item in the list, then a couple of blank lines to make it easier to see. (More on printing [here](_Python%20syntax%20cheat%20sheet.ipynb#The-print()-function).)

In [ ]:
# make a `for` loop to print each row
# plus a couple of blank rows


OK, now we're getting a clearer picture of the data we're targeting. In each row are several `td` tags, which stand for "table data," and they provide various bits of data in this order:
0. Offender number
1. Link to the inmate's detail page
2. Last name
3. First name
4. DOB
5. Gender
6. Race
7. Date inmate went to death row
8. County
9. Date of offense

Why did we start counting with 0? Becuase that's how Python (and other programming languages) start counting -- that will be important here in a second.

So first, a quick recap: Up to this point we have:
- Fetched a web page
- Parsed it into a BeautifulSoup object
- Found the table in the soup
- Found the rows in the table

Our next step is to grab the pieces of data within each row. So as we're looping over the rows, we'll use `find_all()` again to grab the `td` cells and set them equal to variables we can make sense of.

One thing, though -- we want to skip the header row and start with the actual data. In other words, we want to loop over the `rows` list minus the first item in that list, which we can do using a technique called "list slicing." Counting in Python starts with zero, so we want item 1 in that list all the way to the end (skipping item 0, which is the header row). Here's how that works:

In [ ]:
# slice rows to remove header


So instead of this:

```python
for item in rows:
    ...
```

We'll do this:

```python
for item in rows[1:]:
    ...
```

In [ ]:
# for loop but with the header removed


Within each iteration, then, we're going to pull out a list of table data cells and start working with them. Let's start with the first one:

In [ ]:
# start a for loop

    # find all of the `td` tags inside this row

    
    # the inmate ID is in the first [0] cell

    
    # print it


You'll notice that we're getting the entire tag, but we just want the contents. You can access the `text` attribute of that tag to get just the contents:

In [ ]:
# start a for loop

    # find all of the `td` tags inside this row

    
    # the inmate ID is in the first [0] cell
    # just want the text tho

    
    # print it all out


Now we can follow that pattern to get the rest of the data bits (we'll deal with the link in a second):

In [ ]:
# start aa for loop

    # find all of the `td` tags inside this row

    
    # the inmate ID is in the first [0] cell
    # just want the text tho

    
    # link [1]

    
    # last name [2]


    # first name [3]


    # dob [4]


    # gender [5]


    # race [6]


    # intake datae [7]


    # county [8]


    # offense date [9]

    
    # drop it into a list

    
    # and print that list


The text "offender information" isn't super useful -- it would be better if we grabbed the actual link, instead. So instead of accessing the `.text` attribute of the `td` tag, we're going to grab the `'href'` value of the `a` tag inside it (`a` tags are hyperlinks). N.B.: To get the `href` attribute, we'll use bracket notation `['href']`, not dot notation `.href`.

In [ ]:
# start a for loop

    # find all of the `td` tags inside this row

    
    # the inmate ID is in the first [0] cell
    # just want the text tho

    
    # link [1]

    
    # last name [2]


    # first name [3]


    # dob [4]


    # gender [5]


    # race [6]


    # intake datae [7]


    # county [8]


    # offense date [9]

    
    # drop it into a list

    
    # and print that list


Almost there! The link is relative, and we want a fully qualified URL for our data set. So let's prepend the rest of the URL to each link: `https://www.tdcj.texas.gov/death_row/`. In Python, if you "add" two strings together with a plus sign, it concatenates them:

In [ ]:
# start a for loop

    # find all of the `td` tags inside this row

    
    # the inmate ID is in the first [0] cell
    # just want the text tho

    
    # link [1]

    
    # last name [2]


    # first name [3]


    # dob [4]


    # gender [5]


    # race [6]


    # intake datae [7]


    # county [8]


    # offense date [9]

    
    # drop it into a list

    
    # and print that list


## Your turn

In groups, fetch [the page of Senate press accreditations](https://www.dailypress.senate.gov/?page_id=67) and parse each row in the table of journalists into a list.